## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
import openpathsampling as paths
import openmmtools as omt
import simtk.openmm as omm
import simtk.unit as u
import mdtraj as md

Create Alanine in Vacuum and run it using OPS.

In [2]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [3]:
#! skip
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_positions': simtk.unit.quantity.Quantity,
 '_system': simtk.openmm.openmm.System,
 '_topology': simtk.openmm.app.topology.Topology}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [4]:
template = paths.tools.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [5]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [6]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [7]:
engine = paths.OpenMMEngine(template, system, integrator)

Let's run a simulation of 10 steps.

In [8]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(10).can_append])

In [9]:
print traj

Trajectory[10]


In [10]:
#! skip
print traj[5].coordinates

[[ 0.18766728  0.11599674  0.01245003]
 [ 0.19458343  0.22444926  0.00401419]
 [ 0.14436734  0.27122679  0.08869708]
 [ 0.14886759  0.2574746  -0.08926169]
 [ 0.33994535  0.26638269  0.0060984 ]
 [ 0.4270958   0.18970115  0.04380049]
 [ 0.36886492  0.3899833  -0.03181567]
 [ 0.29336685  0.45243084 -0.05633818]
 [ 0.49859309  0.45657605 -0.0240012 ]
 [ 0.55814373  0.41515264  0.05735517]
 [ 0.57325351  0.43391326 -0.15561192]
 [ 0.51911265  0.48312768 -0.23640619]
 [ 0.67427623  0.473205   -0.14414121]
 [ 0.58423054  0.32728139 -0.17536469]
 [ 0.47495216  0.60669047 -0.00127662]
 [ 0.35958388  0.64924192  0.00474684]
 [ 0.58359271  0.6837917   0.00902627]
 [ 0.67474246  0.64066136  0.00332441]
 [ 0.57127428  0.8289724   0.01853087]
 [ 0.46718934  0.86095464  0.0234778 ]
 [ 0.62214297  0.86301553  0.10872201]
 [ 0.61715895  0.8743428  -0.0693164 ]] nm


In [11]:
psi = md.compute_psi(traj.md())

In [12]:
#! ignore
psi[1][3:8]

array([[ 3.10614586],
       [-3.09586358],
       [ 3.13675618],
       [ 3.12832665],
       [ 3.09317851]], dtype=float32)

In [13]:
st = paths.Storage('engine_store_test.nc', mode='w', template=template)

In [14]:
st.engines.save(engine);

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [15]:
#! skip
st.variables['engines_json'][0][0:256] + '...'

u'{"_cls": "OpenMMEngine", "_dict": {"integrator_xml": "<?xml version=\\"1.0\\" ?>\\n<Integrator constraintTolerance=\\"1e-05\\" stepSize=\\".002\\" type=\\"VerletIntegrator\\" version=\\"1\\"/>\\n", "options": {"timestep": null, "n_frames_max": 5000, "platform": "OpenC...'

In [16]:
st.close()

### try to load the stored engine

In [17]:
st = paths.AnalysisStorage('engine_store_test.nc')

In [18]:
engine = st.engines[0]

Run another 10 steps to test

In [19]:
traj = engine.generate(engine.template, [paths.LengthEnsemble(10).can_append])

In [20]:
print traj

Trajectory[10]
